In [17]:
from temporal_constraint import TemporalConstraint  # Ensure this is defined correctly
from stn import STN  # Ensure this is defined correctly

num_patients = 25

def setup_patient_arrivals():
    stn = STN()
    stn.add_edge('START', 'START', TemporalConstraint([0.0, 0.0]))  # Starting point

    # Constants for the arrival times setup
    initial_delay = 10  # Initial delay for the first patient
    max_time = 10
    increment_per_patient = 2  # Increment time for each subsequent patient within a wave
    wave_increment = 10  # Time increment per wave
    patients_per_wave = 5
    num_waves = num_patients // patients_per_wave

    # Generate edges
    for wave in range(num_waves):
        for i in range(patients_per_wave):
            patient_number = wave * patients_per_wave + i + 1
            patient_id = f'Patient_{patient_number}_arrives'
            min_time = 0
            max_time += increment_per_patient
            stn.add_edge('START', patient_id, TemporalConstraint([min_time, max_time]))
            print(f"Added edge from START to {patient_id} with time window [{min_time}, {max_time}]")

    return stn

hospital_stn = setup_patient_arrivals()

Added edge from START to Patient_1_arrives with time window [0, 12]
Added edge from START to Patient_2_arrives with time window [0, 14]
Added edge from START to Patient_3_arrives with time window [0, 16]
Added edge from START to Patient_4_arrives with time window [0, 18]
Added edge from START to Patient_5_arrives with time window [0, 20]
Added edge from START to Patient_6_arrives with time window [0, 22]
Added edge from START to Patient_7_arrives with time window [0, 24]
Added edge from START to Patient_8_arrives with time window [0, 26]
Added edge from START to Patient_9_arrives with time window [0, 28]
Added edge from START to Patient_10_arrives with time window [0, 30]
Added edge from START to Patient_11_arrives with time window [0, 32]
Added edge from START to Patient_12_arrives with time window [0, 34]
Added edge from START to Patient_13_arrives with time window [0, 36]
Added edge from START to Patient_14_arrives with time window [0, 38]
Added edge from START to Patient_15_arrives

In [ ]:
from drea import drea

In [ ]:
drea(hospital_stn, dispatcher)

000.0000: Starting
012.0908: Dispatched Patient_1_arrives 


In [12]:
for i in range(1, num_patients + 1):
    arrival_node = f'Patient_{i}_arrives'
    transport_node = f'Patient_{i}_transported_to_bed'
    # Transportation can happen immediately after arrival but has a window up to 10 minutes later
    hospital_stn.add_edge(arrival_node, transport_node, TemporalConstraint([5, 15]))
    print(f"Added edge from {arrival_node} to {transport_node} with time window [0.0, 10.0]")

Added edge from Patient_1_arrives to Patient_1_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_2_arrives to Patient_2_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_3_arrives to Patient_3_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_4_arrives to Patient_4_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_5_arrives to Patient_5_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_6_arrives to Patient_6_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_7_arrives to Patient_7_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_8_arrives to Patient_8_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_9_arrives to Patient_9_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_10_arrives to Patient_10_transported_to_bed with time window [0.0, 10.0]
Added edge from Patient_11_arrives to Patient_11

In [13]:
hospital_stn.stn.nodes

NodeView(('START', 'Patient_1_arrives', 'Patient_2_arrives', 'Patient_3_arrives', 'Patient_4_arrives', 'Patient_5_arrives', 'Patient_6_arrives', 'Patient_7_arrives', 'Patient_8_arrives', 'Patient_9_arrives', 'Patient_10_arrives', 'Patient_11_arrives', 'Patient_12_arrives', 'Patient_13_arrives', 'Patient_14_arrives', 'Patient_15_arrives', 'Patient_16_arrives', 'Patient_17_arrives', 'Patient_18_arrives', 'Patient_19_arrives', 'Patient_20_arrives', 'Patient_21_arrives', 'Patient_22_arrives', 'Patient_23_arrives', 'Patient_24_arrives', 'Patient_25_arrives', 'Patient_1_transported_to_bed', 'Patient_2_transported_to_bed', 'Patient_3_transported_to_bed', 'Patient_4_transported_to_bed', 'Patient_5_transported_to_bed', 'Patient_6_transported_to_bed', 'Patient_7_transported_to_bed', 'Patient_8_transported_to_bed', 'Patient_9_transported_to_bed', 'Patient_10_transported_to_bed', 'Patient_11_transported_to_bed', 'Patient_12_transported_to_bed', 'Patient_13_transported_to_bed', 'Patient_14_transpor

In [14]:
drea(hospital_stn, dispatcher)

KeyError: 'Patient_1_transported_to_bed'

In [17]:
def setup_healthcare_stn(transported_stn):
    num_patients = 25  # Number of patients

    for i in range(1, num_patients + 1):
        transport_node = f'Patient_{i}_transported_to_bed'
        nurse_arrival_node = f'Nurse_arrives_for_Patient_{i}'
        robot_arrival_node = f'Robot_arrives_for_Patient_{i}'
        screening_start_node = f'Patient_{i}_screening_starts'
        screening_end_node = f'Patient_{i}_screening_ends'

        # Add edges for nurse and robot to arrive after patient is transported to bed
        stn.add_edge(transport_node, nurse_arrival_node, TemporalConstraint([0.0, 15.0]))
        stn.add_edge(transport_node, robot_arrival_node, TemporalConstraint([0.0, 10.0]))

        # Add synchronization edges
        # These edges imply that the screening cannot start until both have arrived
        stn.add_edge(nurse_arrival_node, screening_start_node, TemporalConstraint([0.0, float('inf')]))
        stn.add_edge(robot_arrival_node, screening_start_node, TemporalConstraint([0.0, float('inf')]))

        # Define screening duration
        stn.add_edge(screening_start_node, screening_end_node, TemporalConstraint([5.0, 5.0]))

        print(f"Screening for Patient {i} is set up with nurse and robot arrival dependencies.")

    return stn

# Create the STN for the healthcare scenario
hospital_stn = setup_healthcare_stn(hospital_stn)

Screening for Patient 1 is set up with nurse and robot arrival dependencies.
Screening for Patient 2 is set up with nurse and robot arrival dependencies.
Screening for Patient 3 is set up with nurse and robot arrival dependencies.
Screening for Patient 4 is set up with nurse and robot arrival dependencies.
Screening for Patient 5 is set up with nurse and robot arrival dependencies.
Screening for Patient 6 is set up with nurse and robot arrival dependencies.
Screening for Patient 7 is set up with nurse and robot arrival dependencies.
Screening for Patient 8 is set up with nurse and robot arrival dependencies.
Screening for Patient 9 is set up with nurse and robot arrival dependencies.
Screening for Patient 10 is set up with nurse and robot arrival dependencies.
Screening for Patient 11 is set up with nurse and robot arrival dependencies.
Screening for Patient 12 is set up with nurse and robot arrival dependencies.
Screening for Patient 13 is set up with nurse and robot arrival dependenc

In [31]:
from scipy.stats import norm
from temporal_constraint import TemporalConstraint
from stn import STN

def setup_healthcare_process():
    stn = STN()
    stn.add_edge('START', 'START', TemporalConstraint([0.0, 0.0]))
    num_patients = 25

    # Constants for the arrival times setup
    initial_delay = 10  # Initial delay for the first patient
    max_time = 10
    increment_per_patient = 2  # Increment time for each subsequent patient within a wave
    wave_increment = 10  # Time increment per wave
    patients_per_wave = 5
    num_waves = num_patients // patients_per_wave

    # Setup patient arrivals with increasing time windows
    for wave in range(num_waves):
        for i in range(patients_per_wave):
            patient_number = wave * patients_per_wave + i + 1
            patient_id = f'Patient_{patient_number}_arrives'
            min_time = 0
            max_time += increment_per_patient
            stn.add_edge('START', patient_id, TemporalConstraint([min_time, max_time]))
            print(f"Added edge from START to {patient_id} with time window [{min_time}, {max_time}]")

            # Extend setup to cover the entire patient care process
            transport_to_bed_node = f'Patient_{patient_number}_transported_to_bed'
            nurse_arrival_node = f'Nurse_arrives_for_Patient_{patient_number}'
            robot_arrival_node = f'Robot_arrives_for_Patient_{patient_number}'
            screening_start_node = f'Patient_{patient_number}_screening_starts'
            screening_end_node = f'Patient_{patient_number}_screening_ends'
            scan_begin_node = f'Patient_{patient_number}_scan_begins'
            scan_end_node = f'Patient_{patient_number}_scan_ends'
            doctor_arrival_node = f'Doctor_arrives_for_Patient_{patient_number}'
            doctor_leave_node = f'Doctor_leaves_Patient_{patient_number}'
            transport_to_surgery_node = f'Patient_{patient_number}_surgery_transport'
            surgery_start_node = f'Patient_{patient_number}_surgery'
            surgery_end_node = f'Patient_{patient_number}_surgery_end'
            robot_post_op_node = f'Robot_arrives_for_Patient_{patient_number}_post_op'

            # Define the connections and temporal constraints
            stn.add_edge(patient_id, transport_to_bed_node, TemporalConstraint([5, 15]))
            stn.add_edge(transport_to_bed_node, nurse_arrival_node, TemporalConstraint([0, 15]))
            stn.add_edge(transport_to_bed_node, robot_arrival_node, TemporalConstraint([0, 10]))
            stn.add_edge(nurse_arrival_node, screening_start_node, TemporalConstraint([0, 1000]))
            stn.add_edge(robot_arrival_node, screening_start_node, TemporalConstraint([0, 1000]))
            stn.add_edge(screening_start_node, screening_end_node, TemporalConstraint([5, 5]))
            stn.add_edge(screening_end_node, scan_begin_node, TemporalConstraint([0, 0]))
            stn.add_edge(scan_begin_node, scan_end_node, TemporalConstraint(norm(loc=15, scale=7), contingent=True))
            stn.add_edge(scan_end_node, doctor_arrival_node, TemporalConstraint(norm(loc=4, scale=2), contingent=True))
            stn.add_edge(doctor_arrival_node, doctor_leave_node, TemporalConstraint(norm(loc=5, scale=10), contingent=True))
            stn.add_edge(doctor_leave_node, transport_to_surgery_node, TemporalConstraint([0, 10]))
            stn.add_edge(transport_to_surgery_node, surgery_start_node, TemporalConstraint([5, 5]))
            stn.add_edge(surgery_start_node, surgery_end_node, TemporalConstraint(norm(loc=20, scale=2), contingent=True))
            stn.add_edge(surgery_end_node, robot_post_op_node, TemporalConstraint([0, 10]))

            print(f"Workflow for Patient {patient_number} from arrival to post-operative care is set up.")

    return stn

# Create and setup the healthcare STN
complete_healthcare_stn = setup_healthcare_process()

Added edge from START to Patient_1_arrives with time window [0, 12]
Workflow for Patient 1 from arrival to post-operative care is set up.
Added edge from START to Patient_2_arrives with time window [0, 14]
Workflow for Patient 2 from arrival to post-operative care is set up.
Added edge from START to Patient_3_arrives with time window [0, 16]
Workflow for Patient 3 from arrival to post-operative care is set up.
Added edge from START to Patient_4_arrives with time window [0, 18]
Workflow for Patient 4 from arrival to post-operative care is set up.
Added edge from START to Patient_5_arrives with time window [0, 20]
Workflow for Patient 5 from arrival to post-operative care is set up.
Added edge from START to Patient_6_arrives with time window [0, 22]
Workflow for Patient 6 from arrival to post-operative care is set up.
Added edge from START to Patient_7_arrives with time window [0, 24]
Workflow for Patient 7 from arrival to post-operative care is set up.
Added edge from START to Patient_8

In [32]:
drea(complete_healthcare_stn, dispatcher)

KeyError: 'Patient_1_transported_to_bed'

In [26]:
from scipy.stats import norm
from temporal_constraint import TemporalConstraint
from stn import STN

def setup_healthcare_process():
    stn = STN()
    stn.add_edge('START', 'START', TemporalConstraint([0.0, 0.0]))
    num_patients = 25

    # Setup patient arrivals with increasing time windows
    for i in range(1, num_patients + 1):
        patient_arrival_node = f'Patient_{i}_arrives'
        stn.add_edge('START', patient_arrival_node, TemporalConstraint(norm(loc=10 + i*2, scale=2), contingent=True))

        # Define the subsequent nodes and dependencies for each patient
        transport_to_bed_node = f'Patient_{i}_transported_to_bed'
        nurse_arrival_node = f'Nurse_arrives_for_Patient_{i}'
        doctor_arrival_node = f'Doctor_arrives_for_Patient_{i}'
        screening_end_node = f'Patient_{i}_screening_ends'

        # Nurse and doctor movement
        nurse_next_patient_node = f'Nurse_leaves_for_Patient_{i+1}' if i < num_patients else 'Nurse_finishes_last_patient'
        doctor_next_patient_node = f'Doctor_leaves_for_Patient_{i+1}' if i < num_patients else 'Doctor_finishes_last_patient'

        # Add edges for the patient care sequence
        stn.add_edge(patient_arrival_node, transport_to_bed_node, TemporalConstraint([5, 15]))
        stn.add_edge(transport_to_bed_node, nurse_arrival_node, TemporalConstraint([0, 15]))
        stn.add_edge(transport_to_bed_node, doctor_arrival_node, TemporalConstraint([0, 10]))

        # Synchronization for nurse and doctor departure to next patient
        if i < num_patients:
            stn.add_edge(nurse_arrival_node, nurse_next_patient_node, TemporalConstraint([0, 10000]))
            stn.add_edge(doctor_arrival_node, doctor_next_patient_node, TemporalConstraint([0, 10000]))

            # Connect nurse and doctor from one patient to the next
            next_nurse_arrival_node = f'Nurse_arrives_for_Patient_{i+1}'
            next_doctor_arrival_node = f'Doctor_arrives_for_Patient_{i+1}'
            stn.add_edge(nurse_next_patient_node, next_nurse_arrival_node, TemporalConstraint([0, 10]))
            stn.add_edge(doctor_next_patient_node, next_doctor_arrival_node, TemporalConstraint([0, 10]))

        print(f"Workflow for Patient {i} is set up with nurse and doctor moving to next patient.")

    return stn

# Create and setup the healthcare STN
complete_healthcare_stn = setup_healthcare_process()


Workflow for Patient 1 is set up with nurse and doctor moving to next patient.
Workflow for Patient 2 is set up with nurse and doctor moving to next patient.
Workflow for Patient 3 is set up with nurse and doctor moving to next patient.
Workflow for Patient 4 is set up with nurse and doctor moving to next patient.
Workflow for Patient 5 is set up with nurse and doctor moving to next patient.
Workflow for Patient 6 is set up with nurse and doctor moving to next patient.
Workflow for Patient 7 is set up with nurse and doctor moving to next patient.
Workflow for Patient 8 is set up with nurse and doctor moving to next patient.
Workflow for Patient 9 is set up with nurse and doctor moving to next patient.
Workflow for Patient 10 is set up with nurse and doctor moving to next patient.
Workflow for Patient 11 is set up with nurse and doctor moving to next patient.
Workflow for Patient 12 is set up with nurse and doctor moving to next patient.
Workflow for Patient 13 is set up with nurse and 

In [27]:
drea(complete_healthcare_stn, dispatcher)

KeyError: 'Patient_1_transported_to_bed'